# **Capítulo 2: Novos passos com Python**
Neste capítulo, vamos aprofundar nossos conhecimentos, aprendendo a interagir com arquivos, criar visualizações de dados e manipular datas, que são tarefas essenciais para qualquer análise financeira.

## **2.1 Leitura de arquivos .csv**
A biblioteca **Pandas** é a ferramenta fundamental para ler dados tabulares (como planilhas) em Python. Para ler arquivos no formato `.csv`, usamos a função `pd.read_csv()`.

Por padrão, a função espera que as colunas sejam separadas por vírgula. Se o seu arquivo usar ponto e vírgula (comum em arquivos brasileiros), você pode especificar isso com o argumento `sep=';'`.

In [ ]:
import pandas as pd
import numpy as np

# O código abaixo tentará ler o arquivo "abev3_petr4.csv".
# Se o arquivo não for encontrado, ele criará um DataFrame de exemplo para que o resto do tutorial funcione.

try:
    # Tenta ler o arquivo do disco
    df = pd.read_csv("abev3_petr4.csv")
    print("Arquivo 'abev3_petr4.csv' lido com sucesso!")
except FileNotFoundError:
    print("Arquivo 'abev3_petr4.csv' não encontrado. Criando um DataFrame de exemplo.")
    # Criando dados de exemplo
    datas = pd.date_range(start='2020-01-01', periods=100)
    dados_exemplo = {
        'data': datas,
        'petr4': (np.random.randn(100) * 0.02 + 0.0005).cumsum() + 25,
        'abev3': (np.random.randn(100) * 0.01 + 0.0002).cumsum() + 15
    }
    df = pd.DataFrame(dados_exemplo)
    # Calculando os retornos para usar nas próximas seções
    df['retorno_diario_petr4'] = df['petr4'].pct_change()
    df['retorno_diario_abev3'] = df['abev3'].pct_change()

# Exibindo as 5 primeiras linhas do DataFrame para verificação
df.head()

## **2.2 Gráficos com Matplotlib e Seaborn**

**Matplotlib** é a biblioteca base para visualizações em Python, enquanto **Seaborn** é construída sobre ela, oferecendo uma interface mais simples e gráficos estatísticos mais atraentes. A sintaxe básica envolve chamar uma função de plotagem e passar o DataFrame e as colunas desejadas.

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define o estilo visual dos gráficos para um fundo branco com grades
sns.set_theme(style="whitegrid")

### **2.2.1 Gráfico de Dispersão ou Gráfico de Pontos**

Este gráfico é usado para visualizar a relação entre duas variáveis numéricas.

In [ ]:
# Cria o gráfico de dispersão usando a função scatterplot do Seaborn
sns.scatterplot(data=df, x='petr4', y='abev3')

# Adiciona um título ao gráfico e o exibe
plt.title('Dispersão: Preços PETR4 vs. ABEV3')
plt.show()

### **2.2.2 Gráfico de Linha**

A sintaxe para fazer um gráfico de linha é semelhante à do gráfico de pontos. Contudo, esse tipo de gráfico é mais usado quando a variável do eixo X é uma data, sendo perfeito para visualizar a evolução dos preços de um ativo ao longo do tempo.

In [ ]:
# Cria o gráfico de linha usando a função lineplot do Seaborn
sns.lineplot(data=df, x='data', y='petr4')

# Adiciona um título ao gráfico
plt.title('Evolução do Preço de PETR4')

# Rotaciona os rótulos do eixo X para melhor visualização
plt.xticks(rotation=45)

# Exibe o gráfico
plt.show()

### **2.2.3 Histogramas**

Os histogramas são utilizados para representar a distribuição de dados de uma variável quantitativa em intervalos contínuos. No mercado financeiro, uma aplicação comum de histogramas é analisar a distribuição de retornos diários de um ativo.

In [ ]:
# Cria o histograma dos retornos de PETR4
# O argumento 'bins' controla o número de barras (intervalos)
# kde=True adiciona uma linha de estimativa de densidade sobre o histograma
sns.histplot(data=df, x='retorno_diario_petr4', bins=30, kde=True)

# Adiciona um título e exibe o gráfico
plt.title('Distribuição dos Retornos Diários de PETR4')
plt.show()

## **2.3 Trabalhando com Datas**

Datas são um caso especial em análise de dados. O Pandas possui um tipo de dado específico para elas, o `datetime`, que é extremamente poderoso. A função `pd.to_datetime()` é a principal ferramenta para converter colunas de texto (ou outros formatos) para o tipo `datetime`, o que nos permite realizar cálculos e extrair informações com facilidade.

In [ ]:
# Vamos garantir que nossa coluna 'data' esteja no formato correto
# Primeiro, vemos o tipo de dado atual da coluna
print(f"Tipo de dado ANTES da conversão: {df['data'].dtype}")

# Agora, usamos pd.to_datetime para converter
df['data'] = pd.to_datetime(df['data'])

# Verificamos o tipo novamente para confirmar a mudança
print(f"Tipo de dado DEPOIS da conversão: {df['data'].dtype}")

### **2.3.1 Gerar uma Sequência de Datas**

Para criar um vetor (ou uma lista) de datas sequenciais, usamos a função `pd.date_range()`. Ela é muito flexível e permite especificar o início, o fim, o número de períodos e a frequência (diária, mensal, anual, etc.).

In [ ]:
# Gerar um vetor de datas diárias entre duas datas
datas_diarias = pd.date_range(start="2020-01-01", end="2020-01-20", freq="D")
print("Datas Diárias:")
print(datas_diarias)

# Gerar um vetor de datas mensais (pegando o primeiro dia de cada mês)
datas_mensais = pd.date_range(start="2020-01-01", end="2020-12-01", freq="MS")
print("\nDatas Mensais:")
print(datas_mensais)

# Gerar 7 datas, uma por semana, a partir de uma data inicial
datas_semanais = pd.date_range(start="2020-01-01", periods=7, freq="W")
print("\nDatas Semanais (fim de semana):")
print(datas_semanais)

## **2.4 Convertendo Textos (Strings) para Datas**

Muitas vezes, as datas em um arquivo de dados vêm como texto, e em formatos variados. Aqui no Brasil, por exemplo, usamos o formato `DD/MM/AAAA`.

A função `pd.to_datetime()` é inteligente e consegue interpretar a maioria dos formatos comuns. Para ajudá-la a entender formatos como o brasileiro, onde o dia vem antes do mês, usamos o argumento `dayfirst=True`. A função também consegue interpretar informações de horário (hora, minuto, segundo).

In [ ]:
# Lista de datas em formato texto, com diferentes separadores
datas_brasil = ["01/12/2019", "20/11/2018", "30-01-1990"]

# Convertendo para o formato datetime
# dayfirst=True informa ao Pandas que o primeiro número é o dia
datas_convertidas = pd.to_datetime(datas_brasil, dayfirst=True)
print("Datas convertidas:")
print(datas_convertidas)


# Exemplo com dados de horário (hora, minuto e segundo)
data_hora_texto = "30-09-2019 14:51:39"
data_hora_convertida = pd.to_datetime(data_hora_texto, dayfirst=True)

print("\nData e hora convertidas:")
print(data_hora_convertida)
print(f"Tipo do objeto: {type(data_hora_convertida)}")

## **2.5 Extrair Componentes de uma Data**

Uma vez que uma coluna está no formato `datetime`, podemos facilmente extrair seus componentes (ano, mês, dia, dia da semana, etc.) usando o acessador `.dt`. Ele funciona como um portal para dezenas de propriedades e métodos relacionados a datas.

In [ ]:
# Vamos usar o vetor de datas que convertemos na seção anterior
datas_texto = ["01/12/2019 13:51:15", "20/11/2018 00:00:00", "30/01/1990 08:00:00"]
datas_convertidas = pd.to_datetime(datas_texto, dayfirst=True)

# Para usar o acessador .dt, precisamos que os dados estejam em uma Series do Pandas
datas_series = pd.Series(datas_convertidas)

# Extraindo os componentes:
print(f"Ano: {datas_series.dt.year.to_list()}")
print(f"Mês: {datas_series.dt.month.to_list()}")
print(f"Dia: {datas_series.dt.day.to_list()}")
print(f"Hora: {datas_series.dt.hour.to_list()}")
print(f"Dia da semana (nome): {datas_series.dt.day_name(locale='pt_BR').to_list()}")
print(f"Trimestre: {datas_series.dt.quarter.to_list()}")

## **2.6 Operações Matemáticas com Datas**

Geralmente, estamos interessados em três tipos de operações com datas: adicionar ou subtrair um período, calcular a diferença entre duas datas e arredondar datas para o início ou fim de um período.

### Adicionar ou Subtrair um Período de Tempo

Para adicionar uma quantidade fixa de tempo (como dias, horas ou semanas), usamos o objeto `pd.Timedelta`. Para adições que dependem do calendário (como meses, que têm durações diferentes), usamos `pd.DateOffset`.

In [ ]:
import pandas as pd
from pandas.tseries.offsets import DateOffset

data_inicial = pd.to_datetime("2019-12-01 13:51:15")

# Adicionar 7 dias
print(f"Data inicial + 7 dias: {data_inicial + pd.Timedelta(days=7)}")

# Adicionar 3 meses
print(f"Data inicial + 3 meses: {data_inicial + DateOffset(months=3)}")

# Adicionar 1 ano
print(f"Data inicial + 1 ano: {data_inicial + DateOffset(years=1)}")

### Calcular a Diferença entre Duas Datas

Subtrair duas datas em Python (ou duas colunas de datas no Pandas) é tão simples quanto subtrair números. O resultado é um objeto `Timedelta`, que armazena a diferença em dias, horas, minutos e segundos. Para obter apenas o número de dias, usamos o atributo `.days`.

In [ ]:
data1 = pd.to_datetime("1993-09-01")
data2 = pd.to_datetime("2018-06-24")

# Calcular a diferença
diferenca = data2 - data1
print(f"Objeto de diferença: {diferenca}")

# Extrair apenas o número de dias
dias = diferenca.days
print(f"Diferença em dias: {dias}")

# Converter para outras unidades (aproximado)
print(f"Diferença em semanas: {dias / 7:.2f}")
print(f"Diferença em meses: {dias / 30:.2f}")
print(f"Diferença em anos: {dias / 365:.2f}")

### Arredondar Datas

Podemos "arredondar" uma data para o início (`floor`) ou fim (`ceil`) de um período, como semana, mês ou trimestre.

In [ ]:
datas_texto = ["01/12/2019 13:51:15", "20/11/2018 00:00:00"]
datas_series = pd.to_datetime(datas_texto, dayfirst=True)

# Arredondar para o início da semana (domingo)
print("Início da semana:")
print(datas_series.dt.floor('W'))

# Arredondar para o FIM do mês (um padrão comum)
print("\nFim do mês:")
# Ceil('MS') arredonda para o início do próximo mês, então subtraímos 1 dia
print((datas_series.dt.ceil('MS') - pd.Timedelta(days=1)).dt.date)

# Arredondar para o início da hora
print("\nInício da hora:")
print(datas_series.dt.floor('h'))

## **2.7 Estruturas Condicionais (If-Else)**

A estrutura condicional é bastante intuitiva e usa os operadores lógicos que vimos anteriormente. A ideia é: **se (`if`)** uma condição for verdadeira, execute uma tarefa; **senão (`else`)**, execute uma tarefa diferente. Em Python, quando temos múltiplas condições, usamos o **`elif`** (uma contração de "else if").

Essa estrutura é ideal para operar com variáveis individuais.

In [ ]:
a = 9823

if a >= 10000:
    b = 'VALOR ALTO'
elif a < 10000 and a >= 1000: # "elif" é o "else if"
    b = 'VALOR MÉDIO'
else:
    b = 'VALOR BAIXO'

print(b)

Quando queremos aplicar uma lógica condicional a um vetor inteiro ou a uma coluna de um DataFrame, usar a estrutura `if-else` com um loop seria muito lento. Para isso, usamos funções "vetorizadas" que operam em todos os elementos de uma vez.

A ferramenta mais comum para isso é a função `np.where()` da biblioteca NumPy. Para múltiplas condições (equivalente ao `if-elif-else`), a função `np.select()` é uma ótima opção.

In [ ]:
# Primeiro, precisamos do numpy
import numpy as np

# Exemplo com np.where (para duas condições: if/else)
numeros = np.array([5, 12, 1, 18, 9])
resultado_where = np.where(numeros >= 10, 'Maior ou igual a 10', 'Menor que 10')
print("Resultado com np.where:")
print(resultado_where)


# Exemplo com np.select (para múltiplas condições: if/elif/else)
# Simulando o mesmo exemplo do início, mas com vários números
valores = np.array([15000, 839, 9823, 500])

condicoes = [
    valores >= 10000,
    (valores < 10000) & (valores >= 1000)
]
opcoes = [
    'VALOR ALTO',
    'VALOR MÉDIO'
]

# O default é o que acontece se nenhuma condição for atendida (o 'else')
resultado_select = np.select(condicoes, opcoes, default='VALOR BAIXO')

print("\nResultado com np.select:")
print(resultado_select)

## **2.8 Laços de Repetição (Loops)**

Trata-se de um dos conceitos mais importantes de qualquer linguagem de programação. Loops (ou laços) repetem uma sequência de comandos quantas vezes você desejar.

Supondo que você tenha que ler 400 arquivos de dados. Você vai escrever 400 vezes a função de leitura? Claro que não. Nesse caso, basta fazer um loop para percorrer todos os arquivos da pasta e ler um por um.

O `for` é usado para realizar uma série de ordens para cada item em uma determinada sequência (como uma lista de números ou de nomes de arquivos). Sua sintaxe é bem simples: `for item in sequencia:`.

In [ ]:
# Para cada valor (que chamamos de 'i') dentro da lista [1, 2, 3, 4, 5],
# execute o comando print(i**2).
# Note a indentação (espaçamento) dentro do loop. Ela é obrigatória em Python.

for i in [1, 2, 3, 4, 5]:
    print(i**2)

Também é possível utilizar um loop junto com uma estrutura condicional `if`. No exemplo a seguir, queremos ver todos os números de 1 a 1000 que são divisíveis por 29 e por 3 ao mesmo tempo. Para isso, utilizaremos o operador `%` (módulo), que mostra o resto da divisão. Se o resto for zero, o número é divisível.

In [ ]:
# range(1, 1001) cria uma sequência de números de 1 até 1000
for i in range(1, 1001):
    if (i % 29 == 0) and (i % 3 == 0):
        print(i)

## **2.9 Criando suas Próprias Funções**

Funções "encapsulam" uma sequência de comandos e instruções. É uma estrutura nomeada, que recebe parâmetros para iniciar sua execução e, opcionalmente, retorna um resultado ao final. Até o momento, já usamos diversas funções prontas (`print()`, `type()`, etc.). Agora, vamos aprender a criar as nossas.

Isso é extremamente útil para evitar a repetição de código. Se você precisa executar a mesma análise para 10 ativos diferentes, não precisa escrever o código 10 vezes. Você escreve uma função uma vez e a chama 10 vezes.

Em Python, criamos uma função com a palavra-chave `def`.

In [ ]:
# Vamos criar uma função que, dado uma série de retornos, exibe algumas estatísticas.
# 'x' é o nome do parâmetro que a função espera receber.
def analisar_retorno(retornos):
    """
    Esta função calcula e imprime as principais estatísticas de uma série de retornos.
    Ela remove automaticamente os valores ausentes (NA).
    """
    # Garante que estamos trabalhando com uma cópia limpa, sem valores ausentes
    dados_limpos = retornos.dropna()
    
    media = dados_limpos.mean()
    desvio_padrao = dados_limpos.std()
    mediana = dados_limpos.median()
    
    print(f"Média: {media:.6f}")
    print(f"Desvio Padrão: {desvio_padrao:.6f}")
    print(f"Mediana: {mediana:.6f}")

Agora que a função `analisar_retorno` foi definida, ela está disponível em nossa sessão. Podemos chamá-la quantas vezes quisermos, passando diferentes séries de retornos como argumento.

In [ ]:
# Vamos aplicar a função que criamos às séries de retorno do nosso DataFrame 'df'.

print("--- Análise PETR4 ---")
analisar_retorno(df['retorno_diario_petr4'])

print("\n--- Análise ABEV3 ---")
analisar_retorno(df['retorno_diario_abev3'])

## **2.10 Listas**

Nós já vimos os arrays do NumPy, que são sequências de elementos de um **mesmo tipo**. O Python também possui uma estrutura de dados nativa chamada **lista**, que é mais flexível: ela pode armazenar, literalmente, qualquer tipo de objeto em uma mesma coleção (números, textos, DataFrames, outras listas, etc.).

Listas são criadas com colchetes `[]`, com os elementos separados por vírgula.

In [ ]:
# Vamos criar uma lista com objetos de classes diferentes

# Primeiro, preparamos os objetos que irão para a lista
data_frame_pequeno = df.head()  # Um DataFrame do Pandas
elemento_unico_inteiro = 1
um_valor_nulo = None  # 'None' é o equivalente do NA em Python
vetor_string = ['a', 'b', 'c', 'd', 'e']

# Agora, criamos a lista contendo todos esses objetos
minha_lista = [
    data_frame_pequeno,
    elemento_unico_inteiro,
    um_valor_nulo,
    vetor_string
]

# Vamos conferir o output:
# O Jupyter/VSCode exibe cada elemento de forma organizada
minha_lista

Para extrair um elemento de uma lista, usamos a mesma sintaxe de indexação que já vimos, com colchetes `[]`, lembrando que a contagem sempre começa em zero.

In [ ]:
# Acessando o primeiro elemento da lista (o DataFrame)
primeiro_elemento = minha_lista[0]
print("--- PRIMEIRO ELEMENTO (UM DATAFRAME) ---")
print(primeiro_elemento)


# Acessando o quarto elemento da lista (o vetor de strings)
quarto_elemento = minha_lista[3]
print("\n--- QUARTO ELEMENTO (UM VETOR DE STRINGS) ---")
print(quarto_elemento)